In [2]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests

# Define the fetch price function

def fetch_prices(search_query):
    api_key = 'f24ced1c49ca21f114b224bab53ab0541e61ad8be13136dae648481c43d46edf'  # Replace with your SerpApi API key
    
    try:
        params = {
            'q': search_query,
            'api_key': api_key,
            'engine': 'google',
            'tbm': 'shop'  # Specify the type of search (e.g., shopping results)
        }
          
        response = requests.get('https://serpapi.com/search', params=params)
        data = response.json()
        
        # Extract the product names and prices from the response
        results = []
        for result in data['shopping_results']:
            name = result['title']
            price = result['price']
            results.append({'name': name, 'price': price})
        
        return results
    
    except Exception as e:
        print('Error fetching prices:', e)
        return None

In [4]:
# Input item name for search query and fetch the result

search_query = 'mangoes'  # Item name you want to search for
prices = fetch_prices(search_query)

In [7]:
# Print the price retrieval result

if prices is not None:
    for product in prices:
        print('Product Name:', product['name'])
        print('Price:', product['price'])
        print('---')
else:
    print('Failed to fetch prices.')

Product Name: Fresh Baby Green Mangos 2.2 lb Bag
Price: $6.22
---
Product Name: Fresh Kent Mangoes (9 lbs)
Price: $32.99
---
Product Name: Fresh Mango Fruit, Size: One Size
Price: $0.99
---
Product Name: Fresh Produce Mangoes, 1 Each, Size: 1 lbs, Orange
Price: $0.88
---
Product Name: Mango Fresh ( 5 kgs ie, 12-15 big size Mangos)
Price: $40.50
---
Product Name: Tropical Mango Box - Order Your Fresh Tropical Mangos Online from Tropical Fruit Box
Price: $88.00
---
Product Name: Fresh Champagne Mangoes (Golden Atulfo) - 5 Pounds
Price: $36.95
---
Product Name: Del Monte Mangos, in Light Syrup, Diced - 15 oz
Price: $3.95
---
Product Name: 8 Selected Sweet and Juicy Fresh Large Mango Fruit - 9 lb Pounds
Price: $43.69
---
Product Name: GreatPlentiFulShop Fresh Mangoes 7 Pieces ( 8-10 lbs)
Price: $25.82
---
Product Name: Mangoes | Best Value | 6 Packs Spicy - Mango Pica-Chamoy
Price: $10.20
---
Product Name: Great Value Mango Chunks - 16 oz
Price: $3.32
---
Product Name: Fresh Cut Mango 16 o

In [9]:
!pip install mysql.connector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 55.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247953 sha256=e64d1437356108a34be57d0f29ebcf3c77597a5e9783bd1fdc26bdd32095fd9b
  Stored in directory: /root/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql.connector


In [10]:
# Connect to DiscountMate Database

import mysql.connector as msql
from mysql.connector import Error

conn = msql.connect(host='34.129.54.95', 
                    user='oscar',
                    password='p@ssword0501')

cursor = conn.cursor()
print("Database is connected")
cursor.execute("use dcmdb")
print("Connected to dcmdb")

Database is connected
Connected to dcmdb


In [14]:
if prices is not None:
    for product in prices:
        API_ITEM_NAME = str(product['name'])
        # Replace ' to '' in the product name for the generation of insertion SQL to database 
        API_ITEM_NAME = API_ITEM_NAME.replace("'","''")
        API_ITEM_PRICE = str(product['price'])
        cursor.execute("""
                        INSERT INTO STAGING_API_ITEM_PRICE (
                        COMPANY_NAME,
                        ITEM_NAME,
                        ITEM_PRICE,
                        REC_INSERT_DATE
                        )
                        VALUES (
                        NULL,
                        \'""" + API_ITEM_NAME + """\',
                        \'""" + API_ITEM_PRICE + """\',
                        CURDATE()                     
                        )
                        """)
        print("Insert into STAGING_API_ITEM_PRICE. Item Name: " + API_ITEM_NAME + " | Price: " + API_ITEM_PRICE)
        conn.commit()
else:
    print('Failed to fetch prices.')

Insert into STAGING_API_ITEM_PRICE. Item Name: Fresh Baby Green Mangos 2.2 lb Bag | Price: $6.22
Insert into STAGING_API_ITEM_PRICE. Item Name: Fresh Kent Mangoes (9 lbs) | Price: $32.99
Insert into STAGING_API_ITEM_PRICE. Item Name: Fresh Mango Fruit, Size: One Size | Price: $0.99
Insert into STAGING_API_ITEM_PRICE. Item Name: Fresh Produce Mangoes, 1 Each, Size: 1 lbs, Orange | Price: $0.88
Insert into STAGING_API_ITEM_PRICE. Item Name: Mango Fresh ( 5 kgs ie, 12-15 big size Mangos) | Price: $40.50
Insert into STAGING_API_ITEM_PRICE. Item Name: Tropical Mango Box - Order Your Fresh Tropical Mangos Online from Tropical Fruit Box | Price: $88.00
Insert into STAGING_API_ITEM_PRICE. Item Name: Fresh Champagne Mangoes (Golden Atulfo) - 5 Pounds | Price: $36.95
Insert into STAGING_API_ITEM_PRICE. Item Name: Del Monte Mangos, in Light Syrup, Diced - 15 oz | Price: $3.95
Insert into STAGING_API_ITEM_PRICE. Item Name: 8 Selected Sweet and Juicy Fresh Large Mango Fruit - 9 lb Pounds | Price: $

In [15]:
conn.close()
print('Closed connection to DiscountMate database')

Closed connection to DiscountMate database
